# Import libraries

In [3]:
import numpy as np;

import time
import matplotlib.pyplot as plt 
plt.rc("font", size=16)

import seaborn as sns;

import plotly.graph_objects as go

from datetime import datetime;
__timestamp = None;

from scipy.spatial import ConvexHull
from scipy.interpolate import griddata


def tic():
    global __timestamp;
    __timestamp = datetime.now();
    
def toc():
    global __timestamp;
    print('Time elapsed: %.2f seconds' % (datetime.now() - __timestamp).total_seconds());
    
pics_dir = '/home/madman/Pictures/tmp/'

# 2D convex hull using Graham scan

In [4]:
def graham_scan_f(x, fx):
    if len(x) <= 2:
        return x, fx;
    
    if len(x) != len(fx):
        print(x)
        print(fx)
        raise Exception('X and F(x) sizes does not match!')
        
    stack = [];

    stack.append((x[0], fx[0]));
    stack.append((x[1], fx[1]));

    for i in range(2, len(x)):
        while len(stack) > 1 and (stack[-1][0]-stack[-2][0])*(fx[i]-stack[-1][1]) - (stack[-1][1]-stack[-2][1])*(x[i]-stack[-1][0]) < 0:
            stack.pop();
        stack.append((x[i], fx[i]));

    conv_x = np.array(list(zip(*stack))[0], dtype = x.dtype);
    conv_f = np.array(list(zip(*stack))[1], dtype = x.dtype);

    return conv_x, conv_f;


# Jarvis

In [5]:
def gift_wrapping_f(x, fx):

    if len(x) <= 2:
        return x, fx;
    
    stack = []
    N = len(x)
    
    PointsMatrix = np.array([x,fx]).T

    p0 = np.array([x[0], fx[0] + 1])
    PointsMatrix = np.concatenate(([p0], PointsMatrix), axis = 0)
    p = np.array([x[0], fx[0]])
    stack.append(p0)
    stack.append(p)
    k = 1
    
    while (k != N):
        k = k+1
        aVec = p - stack[-2]
        bVecMatr = PointsMatrix[k:, :] - p
        CosAngles = np.sum(aVec * bVecMatr, 1) / ((bVecMatr[:,0]**2 + bVecMatr[:,1]**2) * (aVec[0] ** 2 + aVec[1] ** 2))**0.5
        k = len(CosAngles) - 1 - np.argmax(np.flip(CosAngles, 0)) + k
        p = PointsMatrix[k]
        stack.append(p)
    
    del stack[0]
    conv_x = np.array(list(zip(*stack))[0], dtype = x.dtype);
    conv_f = np.array(list(zip(*stack))[1], dtype = x.dtype);
    
    return conv_x, conv_f;
    

In [6]:
def payoff_call_cash_or_nothing(price, strike):
    res = 0*price;
    res[price >= strike] = 1.0;
    return res;

def payoff_call_asset_or_nothing(price, strike):
    res = 0*price;
    res[price >= strike] = price[price >= strike];
    return res;

def payoff_call_vanilla(price, strike):
    res = np.fmax(price-strike, 0*price);
    return res;

# 2D algorithms comparisson

In [7]:
def get_option_price_fast(payoff_function, conv_function, N, XstartArray, K, u, d):
    
    ftype = np.float64;
    gN = payoff_function;

    VArray = np.ndarray(shape=(XstartArray.shape[0],), dtype=ftype);

    for i in range(XstartArray.shape[0]):
    
        Xstart = XstartArray[i];

        # generate grids
        Xgrid = [];

        for j in range(N):

            if j == 0:
                grid = [Xstart];
            else:
                grid = [x*d for x in grid] + [u*grid[-1]];

            Xgrid.append(np.array(grid, dtype=ftype));

        # calculate option value on Xgrid
        V = [];

        for n in reversed(range(N)):

            if n == N-1:
                V.insert(0,gN(Xgrid[n], K));
            else:
                conv_x, conv_f = conv_function(Xgrid[n+1],-V[0]);
                V.insert(0, np.interp(Xgrid[n], conv_x, -conv_f));
                
        VArray[i] = V[0][0];

    return VArray;


In [12]:
N = 300

TimeVec_Jarvis = []
TimeVec_Graham = []


payoff_function = payoff_call_vanilla
K = 3.0;
u = 1.002;
d = 0.998;
stepsVec = [5, 10, 20, 50, 100, 200]
V1 = []
V2 = []

for steps in stepsVec:
    
    Xstart = np.linspace(1.5, 5, N);
    
    ##### jarvis
    T_start = time.clock()
    
    V1.append(get_option_price_fast(payoff_function, gift_wrapping_f, steps, Xstart, K, u, d))
        
    T = (time.clock() - T_start) / averaging
    TimeVec_Jarvis.append(T)
    
    #### graham
    T_start = time.clock()
    
    V2.append(get_option_price_fast(payoff_function, graham_scan_f, steps, Xstart, K, u, d))
    
    T = (time.clock() - T_start) / averaging
    TimeVec_Graham.append(T)
    

    
print('Computed successfully')
    
    

/home/madman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning:

time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead

/home/madman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning:

time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead

/home/madman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning:

time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead

/home/madman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning:

time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead



Computed successfully


In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = stepsVec, y = TimeVec_Jarvis, mode = 'lines', name = 'Jarvis'))
fig.add_trace(go.Scatter(x = stepsVec, y = TimeVec_Graham, mode = 'lines', name = 'Graham'))
fig.update_layout(xaxis_title = 'steps', yaxis_title = 'time, s',
                 font = dict(size = 13))
fig.show()

In [14]:
fig.write_image(pics_dir + 'comparisson_vanilla.eps')

In [15]:
fig = go.Figure()
for i in range(6):
    fig.add_trace(go.Scatter(x = Xstart, y = V2[i], mode = 'lines', name = 'steps = {}'.format(stepsVec[i]) ))

fig.update_layout(xaxis_title = 'Asset start price', yaxis_title = 'Option price',
                  font = dict(size = 13))

#fig.update_xaxes(range=[2.8, 3.2])
fig.show()

In [16]:
fig.write_image(pics_dir + 'comparisson_v_1.eps')

In [17]:
fig = go.Figure()
for i in range(5, 6):
    fig.add_trace(go.Scatter(x = Xstart, y = V2[i], mode = 'lines', name = 'steps = {}'.format(stepsVec[i]) ))

fig.add_trace(go.Scatter(x = Xstart, y = payoff_function(Xstart, K), mode = 'lines',
                         name = 'g_N(x)'))


fig.update_layout(xaxis_title = 'Asset start price', yaxis_title = 'Option price',
                  font = dict(size = 13))

#fig.update_xaxes(range=[2.8, 3.2])
fig.show()

In [18]:
fig.write_image(pics_dir + 'comparisson_v_2.eps')

In [19]:
NVec = np.arange(100, 1000, 100)

TimeVec_Jarvis = []
TimeVec_Graham = []


payoff_function = payoff_call_cash_or_nothing
K = 3.0;
u = 1.002;
d = 0.998;
steps = 50
V1 = []
V2 = []

for N in NVec:
    
    Xstart = np.linspace(1.5, 3.5, N);
    
    ##### jarvis
    T_start = time.clock()
    
    V1.append(get_option_price_fast(payoff_function, gift_wrapping_f, steps, Xstart, K, u, d))
        
    T = (time.clock() - T_start) / averaging
    TimeVec_Jarvis.append(T)
    
    #### graham
    T_start = time.clock()
    
    V2.append(get_option_price_fast(payoff_function, graham_scan_f, steps, Xstart, K, u, d))
    
    T = (time.clock() - T_start) / averaging
    TimeVec_Graham.append(T)
    

    
print('Computed successfully')
    
    

/home/madman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning:

time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead

/home/madman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning:

time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead

/home/madman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning:

time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead

/home/madman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning:

time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead



Computed successfully


In [20]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = NVec, y = TimeVec_Jarvis, mode = 'lines', name = 'Jarvis'))
fig.add_trace(go.Scatter(x = NVec, y = TimeVec_Graham, mode = 'lines', name = 'Graham'))
fig.update_layout(xaxis_title = 'M, points', yaxis_title = 'time, s',
                 font = dict(size = 13))
fig.show()

In [21]:
fig.write_image(pics_dir + 'comparisson_v_M.eps')

# 3D convex hull via Discrete Legendre Transform

In [22]:
def conjugate1D(S, fX, X, minusflag=False):
    a = float(-1.0 if minusflag else 1.0);
    if hasattr(S,'__iter__'):
        return a * np.array([np.max(s*X - fX) for s in S], dtype=S.dtype);
    else:
        return a* np.max(S*X - fX);
    
def Legendre_conv(Xgrid, Ygrid, fgrid):
    X = Xgrid[0,:]
    Y = Ygrid[:,0]
    DXgrid_size = X.shape[0];
    DYgrid_size = Y.shape[0];

    f_conjY = np.zeros(shape=(DXgrid_size, Y.shape[0],), dtype=X.dtype);
    f_conj = np.zeros(shape=(DXgrid_size, DYgrid_size,), dtype=X.dtype);


    DXgrid_min = np.inf;
    DXgrid_max = -np.inf;
    for j in range(Y.shape[0]):
        conv_x, conv_f = graham_scan_f(X, fgrid[:,j]);
        S = np.diff(conv_f) / np.diff(conv_x);
        DXgrid_min = min(DXgrid_min, S[0]);
        DXgrid_max = max(DXgrid_max, S[-1]);

    DX_grid = np.linspace(start=DXgrid_min, stop=DXgrid_max, num=DXgrid_size);

    for j in range(Y.shape[0]):
        f_conjY[:,j] = conjugate1D(DX_grid, fgrid[:,j], X, minusflag=True);


    DYgrid_min = np.inf;
    DYgrid_max = -np.inf;
    for i in range(DX_grid.shape[0]):
        conv_x, conv_f = graham_scan_f(Y, f_conjY[i,:]);
        S = np.diff(conv_f) / np.diff(conv_x);
        DYgrid_min = min(DYgrid_min, S[0]);
        DYgrid_max = max(DYgrid_max, S[-1]);

    DY_grid = np.linspace(start=DYgrid_min, stop=DYgrid_max, num=DYgrid_size);

    for i in range(DX_grid.shape[0]):
        f_conj[i,:] = conjugate1D(DY_grid, f_conjY[i,:], Y, minusflag=False);


    f_conv = np.zeros(shape=fgrid.shape, dtype=X.dtype);
    f_convY = np.zeros(shape=(X.shape[0],DY_grid.shape[0]), dtype=X.dtype);

    for j in range(DY_grid.shape[0]):
        f_convY[:,j] = conjugate1D(X, f_conj[:,j], DX_grid, minusflag=True);

    for i in range(X.shape[0]):
        f_conv[i,:] = conjugate1D(Y, f_convY[i,:], DY_grid, minusflag=False);

    return f_conv



In [23]:
tic();

X = np.linspace(start=-2.3, stop=2.3, num=50, dtype=float);
Y = np.linspace(start=-2.3, stop=2.3, num=50, dtype=float);
f = lambda x,y: np.sin(x*x + y*y) + 0.5*(np.abs(x)**2+np.abs(y)**2);


Xgrid, Ygrid = np.meshgrid(X,Y);
fgrid = f(Xgrid, Ygrid);

f_conv = Legendre_conv(Xgrid, Ygrid, fgrid)

toc(); 
#print('grid size: %d x %d' % (X.shape[0], Y.shape[0]));

#f_conv


Time elapsed: 0.33 seconds


In [24]:
Xgrid, Ygrid = np.meshgrid(X,Y);

fig = go.Figure(data = [go.Surface(x = Xgrid, y = Ygrid, z = f(Xgrid, Ygrid)),
                        go.Surface(x = Xgrid, y = Ygrid, z = f_conv)])
fig.update_layout(title='2D convex Hull. Legendre.', autosize=False,
                  width=800, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

# QuickHull 3D

In [25]:
def quick_conv(XGrid, YGrid, fgrid):
    
    max_val = np.max(fgrid)
    
    Xgrid = XGrid
    Ygrid = YGrid
    
    Xgrid = np.concatenate((Xgrid[:, 0].reshape([-1,1]), Xgrid, Xgrid[:, -1].reshape([-1,1])), axis = 1)
    Xgrid = np.concatenate((Xgrid[0, :].reshape([1, -1]), Xgrid, Xgrid[-1 :].reshape([1, -1])), axis = 0)

    Ygrid = np.concatenate((Ygrid[:, 0].reshape([-1,1]), Ygrid, Ygrid[:, -1].reshape([-1,1])), axis = 1)
    Ygrid = np.concatenate((Ygrid[0, :].reshape([1, -1]), Ygrid, Ygrid[-1 :].reshape([1, -1])), axis = 0)

    fgrid = np.concatenate((fgrid[:, 0].reshape([-1,1]), fgrid, fgrid[:, -1].reshape([-1,1])), axis = 1)
    f_values = np.concatenate((fgrid[0, :].reshape([1, -1]), fgrid, fgrid[-1 :].reshape([1, -1])), axis = 0)

    N = Xgrid.shape[0]

    f_values[0, :] = max_val + 1
    f_values[-1,:] = max_val + 1
    f_values[:, 0] = max_val + 1
    f_values[:,-1] = max_val + 1

    x = Xgrid.flatten()
    y = Ygrid.flatten()
    z = f_values.flatten()

    points = np.array([x, y, z]).T
    hull = ConvexHull(points)
    Ind = hull.vertices
    Ind = np.array(Ind)
    Ind = Ind[(Ind > N) & (Ind < N*(N-1)) & (Ind % N != 0) & ((Ind + 1) % N != 0)]

    xx = Xgrid.flat[Ind]
    yy =Ygrid.flat[Ind]
    zz = f_values.flat[Ind].T
    ps = np.array([xx, yy]).T
    f_conv = griddata(ps, zz, (XGrid, YGrid), method='linear')
    return f_conv


In [26]:
N = 50
tic()
X = np.linspace(start=-2.3, stop=2.3, num=N, dtype=float);
Y = np.linspace(start=-2.3, stop=2.3, num=N, dtype=float);
f = lambda x,y: np.sin(x*x + y*y) + 0.5*(np.abs(x)**2+np.abs(y)**2);


Xgrid, Ygrid = np.meshgrid(X,Y)
fgrid = f(Xgrid, Ygrid)

f_conv = quick_conv(Xgrid, Ygrid, fgrid)
toc()

Time elapsed: 0.08 seconds


In [27]:
fig = go.Figure(data = [go.Surface(x = Xgrid, y = Ygrid, z = f(Xgrid, Ygrid)),
                        go.Surface(x = Xgrid, y = Ygrid, z = f_conv)])
fig.update_layout(title='2D convex Hull. Quickhull.', autosize=False,
                  width=800, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

# 3D methods comparisson

In [28]:
def payoff_put_on_max(priceX, priceY, K):
    return np.fmax(K - np.fmax(priceX, priceY), np.zeros(priceX.shape))

def payoff_call_on_max(priceX, priceY, K):
    return np.fmax(np.fmax(priceX, priceY) - K, np.zeros(priceX.shape))

In [29]:
Xgrid = np.linspace(0, 5, 50)
Ygrid = Xgrid
Xgrid, Ygrid = np.meshgrid(Xgrid, Ygrid)
K = 3
fig = go.Figure(data = [go.Surface(x = Xgrid, y = Ygrid, z = payoff_put_on_max(Xgrid, Ygrid, K))])
fig.update_layout(title='put on max', autosize=False,
                  width=800, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [30]:
Xgrid = np.linspace(0, 5, 50)
Ygrid = Xgrid
Xgrid, Ygrid = np.meshgrid(Xgrid, Ygrid)
K = 3
fig = go.Figure(data = [go.Surface(contours = {'x':{'show':True}, 'y':{'show':True}},
                                   x = Xgrid, y = Ygrid, z = payoff_call_on_max(Xgrid, Ygrid, K))])
fig.update_layout(title='put on max', autosize=False,
                  width=800, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [31]:
def get_option_price_fast_3D(payoff_function, conv_function, N, XstartArray, YstartArray, K, Xu, Xd, Yu, Yd):
    
    ftype = np.float64;
    gN = payoff_function;

    VArray = np.ndarray(shape=(XstartArray.shape[0], YstartArray.shape[0]), dtype=ftype);

    for i in range(XstartArray.shape[0]):
    
        Xstart = XstartArray[i];

        # generate grids
        XgridVec = [];

        for j in range(N):

            if j == 0:
                grid = [Xstart];
            else:
                grid = [x*Xd for x in grid] + [Xu*grid[-1]];

            XgridVec.append(np.array(grid, dtype=ftype));
        
        for k in range(YstartArray.shape[0]):
            Ystart = YstartArray[k]
            
            YgridVec = []
            
            Xgrid = []
            Ygrid = []
            for j in range(N):

                if j == 0:
                    grid = [Ystart];
                else:
                    grid = [y*Yd for y in grid] + [Yu*grid[-1]];

                YgridVec.append(np.array(grid, dtype=ftype))
                X, Y = np.meshgrid(XgridVec[j], YgridVec[j]);
                Xgrid.append(X)
                Ygrid.append(Y)
                
            V = [];

            for n in reversed(range(N)):

                if n == N-1:
                    V.insert(0,gN(Xgrid[n], Ygrid[n], K));
                else:
                    conv_f = conv_function(Xgrid[n+1], Ygrid[n+1], -V[0]);
                    points = np.concatenate([Xgrid[n+1].reshape((-1, 1)), Ygrid[n+1].reshape((-1, 1))], axis = 1)
                    conv_f = np.array(conv_f.flatten())
                    V.insert(0, griddata(points, -conv_f.T, (Xgrid[n], Ygrid[n]), method = 'linear'));

            VArray[k, i] = V[0][0];

    return VArray;

In [32]:
NVec = [30, 50, 70, 100]
steps = 10
averaging = 1
TimeVec_Legendre = []
TimeVec_Quickhull = []

payoff_function = payoff_put_on_max
K = 3.0;
Xu = 1.01;
Xd = 0.99;
Yu = 1.02
Yd = 0.98
V1 = []
V2 = []

for N in NVec:
    
    Xstart = np.linspace(0.1, 5, N);
    Ystart = np.linspace(0.1, 5, N);
    
    
    #### quickhull
    T_start = time.clock()
    
    V1.append(get_option_price_fast_3D(payoff_function, quick_conv, steps, Xstart, Ystart,
                                       K, Xu, Xd, Yu, Yd))
    
    T = (time.clock() - T_start) / averaging
    TimeVec_Quickhull.append(T)
    
    
    #### legendre
    T_start = time.clock()
    
    V2.append(get_option_price_fast_3D(payoff_function, Legendre_conv, steps, Xstart, Ystart, 
                                       K, Xu, Xd, Yu, Yd))
        
    T = (time.clock() - T_start) / averaging
    TimeVec_Legendre.append(T)
    

    

    
print('Computed successfully')
    
    

/home/madman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning:

time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead

/home/madman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning:

time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead

/home/madman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning:

time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead

/home/madman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: DeprecationWarning:

time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead



Computed successfully


In [33]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = NVec, y = TimeVec_Legendre, mode = 'lines', name = 'Legendre'))
fig.add_trace(go.Scatter(x = NVec, y = TimeVec_Quickhull, mode = 'lines', name = 'Quickhull'))
fig.update_layout(xaxis_title = 'M, points', yaxis_title = 'time, sec',
                 font = dict(size = 13))

fig.show()

In [34]:
fig.write_image(pics_dir + 'comparisson_pom_M.eps')

In [35]:
F = payoff_put_on_max(Xstart, Ystart, K)

In [36]:
fig = go.Figure(data = [go.Surface(
                                   x = Xgrid, y = Ygrid, 
    z = payoff_put_on_max(Xgrid, Ygrid, K), showscale = True, hidesurface=False)])
fig.update_layout(scene = {
            "xaxis": {"title": 'S1'},
            'yaxis': {'title': 'S2'},
            "zaxis": {'title': 'Option price'},
            'camera_eye': {"x": 1.6, "y": 1.6, "z": 1.3},
            "aspectratio": {"x": 1, "y": 1, "z": 1.5}
        }, autosize=False,
                  width=800, height=600,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [37]:
fig.write_image(pics_dir + 'comparisson_pom_2.eps')